In [ ]:
import fitz  # PyMuPDF
import re
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter # initial_text_split 에서 사용
import os
import json
import uuid # llm_based_semantic_chunking_for_dev_reqs 에서 사용

# --- OpenAI API 클라이언트 초기화 (사용자 기존 코드) ---
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. API 키를 설정해주세요.")
client = OpenAI(api_key=api_key)
LLM_MODEL_FOR_PARSING = "gpt-4o" # 목차 파싱 및 요구사항 추출에 사용할 모델

# --- 사용자 기존 함수 정의 부분 (여기에 extract_text_with_page_info, initial_text_split, llm_based_semantic_chunking_for_dev_reqs 가 있어야 함) ---

def extract_text_with_page_info(pdf_path):
    """
    PDF 파일에서 텍스트를 페이지별로 추출하고, 각 페이지 시작에 페이지 번호를 표기합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    document = fitz.open(pdf_path)
    full_text_with_pages = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        # 페이지 마커를 맨 앞에 추가하고, 실제 텍스트 내용과 명확히 구분되도록 줄바꿈 추가
        full_text_with_pages.append(f"---PAGE_START_{page_num+1}---\n{text.strip()}")
    return "\n".join(full_text_with_pages), document.page_count

def initial_text_split(text, chunk_size=4000, chunk_overlap=200):
    """
    텍스트를 LLM이 처리할 수 있는 크기로 초기 분할합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n---PAGE_START_\d+---\n", "\n\n", "\n", ". ", "? ", "! ", " ", ""],
        keep_separator=True # 페이지 마커 유지를 위해 True 권장
    )
    docs = text_splitter.create_documents([text])
    return docs

# llm_based_semantic_chunking_for_dev_reqs 함수 정의 (사용자 스크립트 내용)
# def llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client, llm_model="gpt-4o"): ...
# (이전에 제공된 프롬프트와 로직을 그대로 사용한다고 가정)
def llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client_instance, llm_model="gpt-4o"): # client_instance로 명칭 변경
    final_semantic_chunks = []
    req_id_counter = 0

    system_prompt = """
    당신은 RFP(제안요청서) 문서에서 시스템 요구사항을 전문적으로 분석하고 개발 표준에 맞춰 구조화하는 시니어 비즈니스 분석가입니다.
    사용자가 제공하는 텍스트는 RFP 문서의 특정 섹션이며, 여기서 모든 기능적, 비기능적, 성능, 보안, 데이터, 제약사항 요구사항을 추출해야 합니다.

    각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.

    -   **id**: 고유 식별자. 예를 들어, 기능적 요구사항은 "FUNC-001", 비기능적은 "NFR-001", 보안은 "SEC-001"과 같이 유형 접두사와 일련번호를 결합하여 생성하십시오. (RFP 원문에 ID가 있다면 그것을 우선 사용)
    -   **type**: 요구사항의 유형. 다음 중 하나를 선택: "기능적", "비기능적", "성능", "보안", "데이터", "제약사항", "시스템 장비 구성", "컨설팅", "테스트", "품질", "프로젝트 관리", "프로젝트 지원", "기타". 텍스트에 명시된 내용에 따라 가장 적합한 유형을 선택하십시오. (예: "시스템 장비 구성요구사항", "컨설팅 요구사항" 등 PDF의 분류명 활용)
    -   **description**: 요구사항에 대한 명확하고 간결한 설명. 원본 PDF의 '요구사항 명칭'과 '정의', '상세설명/세부내용'을 종합하여 개발 친화적인 형태로 작성하십시오. 하나의 요구사항은 하나의 독립적인 기능 또는 특성을 나타내야 합니다.
    -   **acceptance_criteria**: 이 요구사항이 충족되었음을 검증할 수 있는 구체적인 테스트 조건이나 결과. 1~2개의 명확한 문장으로 서술하십시오. 원본 텍스트에 직접적인 검증 기준이 없더라도, 요구사항의 내용에 기반하여 합리적으로 추론하여 작성하십시오. (예: "사용자가 [기능]을 수행했을 때, [예상 결과]가 나타난다.")
    -   **priority**: 요구사항의 중요도. 다음 중 하나를 선택: "필수", "높음", "중간", "낮음". (텍스트에 명시되어 있다면 그대로 사용, 아니면 일반적으로 "필수"로 추정)
    -   **responsible_module**: 이 요구사항이 주로 영향을 미치거나 구현될 것으로 예상되는 시스템/애플리케이션의 주요 모듈 또는 영역 (예: "로그인 모듈", "결제 시스템", "관리자 페이지", "데이터베이스"). 텍스트 내용을 기반으로 추론하십시오.
    -   **source_pages**: 이 요구사항이 발견된 원본 PDF 페이지 번호 리스트. 페이지 구분자(`---PAGE_START_N---`)를 참조하여 정확한 페이지 번호를 파싱하십시오.
    -   **raw_text_snippet**: 이 요구사항이 포함된 원본 텍스트 스니펫. 해당 요구사항을 추출하는 데 사용된 원본 문장 또는 단락(예: 표의 해당 행 전체 내용)을 포함하십시오.

    응답은 **JSON 배열 형식으로만** 제공해야 합니다. 만약 요구사항이 없다면 빈 배열 `[]`을 반환하십시오.
    불필요한 서론, 사업 배경, 계약 조건, 제안 지침 등은 요구사항으로 추출하지 마십시오. PDF의 '상세 요구사항' 목록에 있는 구조화된 항목들 위주로 추출해주십시오.
    """

    for i, chunk_doc in enumerate(initial_chunks):
        chunk_text = chunk_doc.page_content
        print(f"--- LLM 처리 중 (요구사항 상세 추출): 청크 {i+1}/{len(initial_chunks)} (길이: {len(chunk_text)}자) ---")

        page_numbers_in_chunk = sorted(list(set(
            int(p) for p in re.findall(r'---PAGE_START_(\d+)---', chunk_text)
        )))

        user_prompt = f"""
        다음은 RFP 문서의 일부입니다. 이 부분에서 모든 시스템 요구사항을 개발자 표준에 맞춰 JSON 형식으로 추출해 주세요.
        주어진 텍스트 내의 "요구사항 고유번호", "요구사항 명칭", "요구사항 분류", "정의", "상세설명/세부내용" 등의 명시적 필드를 최대한 활용하여 JSON 객체를 구성해주십시오.

        --- 텍스트 ---
        {chunk_text}
        """

        try:
            response = client_instance.chat.completions.create(
                model=llm_model,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0
            )

            llm_response_content = response.choices[0].message.content
            try:
                extracted_data_outer = json.loads(llm_response_content)
                extracted_data_list = []
                if isinstance(extracted_data_outer, list):
                    extracted_data_list = extracted_data_outer
                elif isinstance(extracted_data_outer, dict):
                    # 딕셔너리 값 중에 리스트를 찾아 첫 번째 것을 사용 (일반적인 LLM 응답 패턴)
                    for key_in_dict in extracted_data_outer:
                        if isinstance(extracted_data_outer[key_in_dict], list):
                            extracted_data_list = extracted_data_outer[key_in_dict]
                            break
                    if not extracted_data_list: # 그래도 못찾으면 경고
                        print(f"경고: LLM이 JSON 객체를 반환했으나, 그 안에 예상된 요구사항 리스트를 찾지 못했습니다. 객체 키: {list(extracted_data_outer.keys())}")
                else:
                    print(f"경고: LLM으로부터 예상치 못한 JSON 형식 응답 (리스트 또는 객체 내 리스트가 아님). {llm_response_content[:100]}...")
                    continue
            except json.JSONDecodeError as e_inner:
                print(f"LLM 응답 내용 JSON 파싱 오류 (내부 시도): {e_inner}. 응답: {llm_response_content[:500]}...")
                continue

            for req_idx, req in enumerate(extracted_data_list):
                # 'id'가 없거나 비어있으면 생성, 또는 원본 ID 우선 사용
                original_id = req.get('id') # LLM이 원본 ID (ECR-001 등)를 id 필드에 넣어줬길 기대
                if not original_id:
                    req_type_prefix = "REQ"
                    req_type = req.get('type', '기타').strip()
                    if "기능" in req_type: req_type_prefix = "FUNC"
                    elif "비기능" in req_type: req_type_prefix = "NFR"
                    elif "성능" in req_type: req_type_prefix = "PERF"
                    elif "보안" in req_type: req_type_prefix = "SEC"
                    elif "데이터" in req_type: req_type_prefix = "DATA"
                    elif "제약" in req_type: req_type_prefix = "CONST"
                    elif "장비" in req_type: req_type_prefix = "ECR"
                    elif "컨설팅" in req_type: req_type_prefix = "CSR"
                    elif "테스트" in req_type: req_type_prefix = "TER"
                    elif "품질" in req_type: req_type_prefix = "QUR"
                    elif "관리" in req_type: req_type_prefix = "PMR"
                    elif "지원" in req_type: req_type_prefix = "PSR"
                    req_id_counter += 1
                    req['id'] = f"{req_type_prefix}-{req_id_counter:03d}"

                if 'source_pages' not in req or not req['source_pages']:
                    req['source_pages'] = page_numbers_in_chunk if page_numbers_in_chunk else []

                if 'raw_text_snippet' not in req or not req['raw_text_snippet']:
                    req['raw_text_snippet'] = f"청크 {i+1}에서 추출됨. 원본 청크 일부: {chunk_text[:200]}..." # 개선 필요

                req['type'] = req.get('type', '기타')
                req['description'] = req.get('description', '설명 없음')
                req['acceptance_criteria'] = req.get('acceptance_criteria', '해당하는 경우 명시')
                req['priority'] = req.get('priority', '필수')
                req['responsible_module'] = req.get('responsible_module', '미정')
                final_semantic_chunks.append(req)

        except json.JSONDecodeError as e:
            print(f"LLM 응답 JSON 파싱 오류: {e}. 응답: {llm_response_content[:500]}...")
        except Exception as e:
            print(f"LLM API 호출 또는 처리 중 오류 발생: {e}")

    return final_semantic_chunks


# --- LLM으로 목차 파싱하는 함수 (이전 답변에서 제공 및 개선된 버전) ---
def parse_toc_with_llm(toc_raw_text, client_instance, llm_model="gpt-4o"): # client_instance로 명칭 변경
    """
    LLM을 사용하여 목차 원문 텍스트를 파싱하여 구조화된 목차 항목을 추출합니다.
    """
    system_prompt = """
    당신은 PDF 문서에서 추출된 목차(Table of Contents)의 원시 텍스트를 분석하는 전문가입니다.
    주어진 텍스트를 파싱하여 각 목차 항목의 제목, 페이지 번호, 그리고 해당 항목이 '요구사항' 관련 내용을 담고 있을 가능성을 분석하여 JSON 형태로 반환해야 합니다.
    JSON의 최상위 레벨은 "toc_entries"라는 키를 가진 객체여야 하고, 그 키의 값은 목차 항목 객체들의 리스트여야 합니다.
    """
    user_prompt = f"""
    다음은 PDF에서 추출한 목차로 추정되는 텍스트입니다:

    --- 목차 원문 텍스트 시작 ---
    {toc_raw_text}
    --- 목차 원문 텍스트 끝 ---

    이 텍스트를 분석하여 JSON 객체를 반환해주세요. 이 객체는 "toc_entries"라는 키를 가져야 하며,
    이 키의 값은 각 목차 항목을 나타내는 객체들의 리스트여야 합니다.
    각 목차 항목 객체는 다음 키를 가져야 합니다:
    - "title": (문자열) 목차 항목의 전체 제목. 제목 앞의 번호(예: "1.", "II.", "가.")도 포함해주세요.
    - "page": (정수) 해당 항목의 시작 페이지 번호.
    - "is_requirement_related": (불리언) 제목이나 내용을 볼 때, 해당 항목이 '요구사항', '과업 범위', '제안 요청 상세', '기능 명세', '기술 요건' 등과 관련된 내용을 다룰 가능성이 높으면 true, 그렇지 않으면 false로 설정해주세요.

    예시 JSON 출력 형식:
    {{
      "toc_entries": [
        {{
          "title": "1. 사업 개요",
          "page": 5,
          "is_requirement_related": false
        }},
        {{
          "title": "III. 제안요청 내용",
          "page": 6,
          "is_requirement_related": true
        }},
        {{
          "title": "3. 상세 요구사항",
          "page": 11,
          "is_requirement_related": true
        }},
        {{
          "title": "* 보안 요구사항 별표",
          "page": 63,
          "is_requirement_related": true
        }}
      ]
    }}

    만약 주어진 텍스트가 유효한 목차로 보이지 않거나 항목을 전혀 파싱할 수 없다면,
    "toc_entries" 키의 값으로 빈 리스트 `[]`를 포함하는 JSON 객체를 반환해주세요. (예: {{"toc_entries": []}})
    """
    llm_response_content = "" # 오류 발생 시 로깅을 위해 미리 선언
    try:
        response = client_instance.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0
        )
        llm_response_content = response.choices[0].message.content
        # print(f"DEBUG: LLM RAW RESPONSE FOR TOC: {llm_response_content}")

        parsed_data = json.loads(llm_response_content)

        extracted_list = []
        if isinstance(parsed_data, dict) and "toc_entries" in parsed_data and isinstance(parsed_data["toc_entries"], list):
            extracted_list = parsed_data["toc_entries"]
        else:
            print(f"LLM 응답이 예상된 'toc_entries' 리스트를 포함하는 객체 형식이 아닙니다. 응답: {llm_response_content[:200]}")
            return [] # 빈 리스트 반환

        valid_entries = []
        for entry in extracted_list:
            if isinstance(entry, dict) and 'title' in entry and 'page' in entry:
                try:
                    entry['page'] = int(entry['page'])
                    entry['is_requirement_related'] = bool(entry.get('is_requirement_related', False)) # 명확히 불리언으로
                    valid_entries.append(entry)
                except ValueError:
                    print(f"경고: 페이지 번호 '{entry.get('page')}'를 정수로 변환할 수 없습니다. 항목 건너뜀: {entry.get('title')}")
                    continue
            else:
                print(f"경고: 필수 키(title, page)가 누락된 항목입니다. 건너뜀: {entry}")

        return valid_entries

    except json.JSONDecodeError as e:
        print(f"LLM 목차 파싱 응답 JSON 파싱 오류: {e}. 응답 미리보기: {llm_response_content[:500]}...")
        return []
    except Exception as e:
        print(f"LLM API 호출 또는 처리 중 오류 발생 (목차 파싱): {e}")
        return []

# --- 새로운 헬퍼 함수들 ---
def extract_text_for_pages(full_text_with_pages, start_page_num, end_page_num):
    """
    전체 텍스트에서 특정 페이지 범위의 텍스트만 추출합니다.
    페이지 마커 '---PAGE_START_N---'를 사용합니다.
    """

    # 페이지 범위 유효성 검사 (end_page_num이 start_page_num보다 작을 수 없음)
    if end_page_num < start_page_num:
        # print(f"경고: 끝 페이지({end_page_num})가 시작 페이지({start_page_num})보다 작습니다. 빈 텍스트를 반환합니다.")
        return ""

    # 패턴 구성: 시작 페이지부터 (끝 페이지 + 1) 직전까지 모든 내용을 포함
    # re.escape를 사용하여 페이지 마커의 특수 문자를 이스케이프할 필요는 여기서는 없음
    # DOTALL 플래그를 사용하여 \n도 .에 매치되도록 함

    # 시작점 찾기
    start_marker = f"---PAGE_START_{start_page_num}---"
    start_match = re.search(re.escape(start_marker), full_text_with_pages) # 마커 자체를 찾아야 함

    if not start_match:
        # print(f"경고: 시작 마커 '{start_marker}'를 찾을 수 없습니다.")
        return ""

    text_from_start_page = full_text_with_pages[start_match.start():]

    # 끝점 찾기 (다음 페이지 마커 ---PAGE_START_{end_page_num + 1}---)
    # end_page_num이 문서의 마지막 페이지일 수도 있으므로, end_page_num + 1 마커가 없을 수 있음
    end_marker_exclusive = f"---PAGE_START_{end_page_num + 1}---"
    end_match = re.search(re.escape(end_marker_exclusive), text_from_start_page)

    if end_match:
        return text_from_start_page[:end_match.start()]
    else:
        # end_page_num + 1 마커를 찾지 못하면, start_page_num부터 문서 끝까지 반환 (또는 start_page_num ~ end_page_num의 마지막 내용까지)
        # 이 경우는 end_page_num이 마지막 페이지이거나, 그 이후 페이지 마커가 없는 경우.
        # 좀 더 정확하게 하려면, end_page_num까지의 모든 내용을 가져와야 함.
        # 그러나 페이지 마커 구조상, end_page_num의 내용은 end_page_num+1 마커 전까지임.
        # 따라서 end_match가 없으면 text_from_start_page 전체가 해당 범위임.
        return text_from_start_page


def get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages):
    """
    LLM으로 파싱된 목차에서 '상세 요구사항' 및 '보안 요구사항' 관련 섹션 정보를 추출합니다.
    """
    target_sections = []

    # 페이지 번호 기준으로 목차 정렬 (LLM이 순서대로 안 줄 수도 있으므로)
    sorted_toc = sorted(parsed_toc_entries, key=lambda x: x.get('page', 0))

    # 주요 키워드 - 사용자가 제공한 PDF 목차 기반
    # "III. 제안요청 내용" 안에 "3. 상세 요구사항"이 있으므로, "상세 요구사항"을 찾는 것이 더 정확함.
    keywords_to_find = {
        "상세 요구사항": {"min_pages": 5}, # 최소한 이정도는 될 것이라는 기대
        "보안 요구사항 별표": {"min_pages": 1} # 별표는 짧을 수도 있음
        # 필요시 다른 주요 섹션 키워드 추가 가능
    }

    for i, entry in enumerate(sorted_toc):
        entry_title = entry.get('title', '').strip()
        entry_page = entry.get('page', 0)

        for keyword, props in keywords_to_find.items():
            if keyword in entry_title:
                start_page = entry_page
                end_page = total_pages # 기본값: 문서 끝까지

                # 다음 목차 항목의 시작 페이지 - 1을 현재 섹션의 끝 페이지로 설정
                # 단, 다음 항목이 현재 항목과 같은 페이지에서 시작하면 안됨 (하위 항목일 수 있으므로)
                # 좀 더 정교한 로직: 다음 '주요' 항목을 찾아야 함.
                # 여기서는 일단 다음 항목의 시작 페이지를 사용.
                # 만약 다음 항목이 현재 항목의 하위 항목처럼 보이면 (예: "3. 상세 요구사항" 다음 "3.1 기능 요구사항")
                # 그 하위 항목의 범위를 포함하도록 확장하거나, 아니면 정말 다음 *다른* 주요 섹션까지 봐야 함.
                # 현재 LLM 프롬프트는 is_requirement_related로 판단하므로, 이를 우선적으로 신뢰.

                # 다음 'is_requirement_related=False'인 섹션 또는 다음 주요 섹션(로마숫자/대문자 알파벳 등)을 찾아 end_page 설정
                # 또는 단순히 다음 목차 항목의 시작 페이지 - 1 로 설정
                next_major_section_page = total_pages + 1 # 충분히 큰 값
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page : # 현재 섹션 이후의 페이지만 고려
                        # 여기서 '주요 섹션'을 판단하는 기준이 필요함 (예: 로마 숫자, 대문자 번호 등)
                        # 또는 is_requirement_related=False 인 첫번째 섹션
                        # 또는 단순히 다음 목차 항목
                        next_major_section_page = next_entry_page
                        break

                end_page = min(total_pages, next_major_section_page -1)

                # 시작 페이지와 끝 페이지 유효성 확보
                if end_page < start_page:
                    end_page = start_page

                # 페이지 수가 너무 적으면 해당 섹션 전체를 포함 (예: 11페이지로 나왔는데 실제로는 62까지일 수 있음)
                # 이 부분은 heuristic이므로 주의. 더 좋은 방법은 LLM에게 섹션의 끝을 명확히 묻는 것.
                # 여기서는 min_pages 이상은 되어야 유의미하다고 가정.
                # if (end_page - start_page + 1) < props.get("min_pages", 1) and (start_page + props.get("min_pages", 1) -1) <= total_pages :
                #     pass # end_page = start_page + props.get("min_pages", 1) -1 # 최소 페이지 강제는 위험할 수 있음

                target_sections.append({
                    'title': entry_title,
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 파싱 목차 기반: '{entry_title}' 섹션 (페이지 {start_page}-{end_page}) 식별")
                # 찾은 키워드는 중복 추가 방지 (더 구체적인 항목이 먼저 찾아지도록 keywords_to_find 순서 중요)
                break

    # 만약 아무것도 못찾았지만, is_requirement_related=True 인 항목들이 있다면 그것들을 사용
    if not target_sections:
        print("키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...")
        for i, entry in enumerate(sorted_toc):
            if entry.get('is_requirement_related'):
                start_page = entry.get('page',0)
                end_page = total_pages
                # 위와 동일한 로직으로 end_page 계산
                next_major_section_page = total_pages + 1
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page:
                        if not next_entry.get('is_requirement_related', False): # 다음 주요 섹션이 요구사항 관련이 아니면
                            next_major_section_page = next_entry_page
                            break
                        # 또는 다음 항목이 현재 항목보다 상위 레벨이면 (예: "3.1" 다음 "4.")
                        # 이 부분은 제목의 번호 체계를 분석해야 해서 복잡함. LLM의 is_requirement_related를 신뢰.

                end_page = min(total_pages, next_major_section_page - 1)
                if end_page < start_page: end_page = start_page

                target_sections.append({
                    'title': entry.get('title', '요구사항 관련 섹션'),
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 'is_requirement_related' 플래그 기반: '{entry.get('title')}' 섹션 (페이지 {start_page}-{end_page}) 식별")

    # 중복 제거 및 병합 (예: "III. 제안요청 내용"과 그 하위 "3. 상세 요구사항"이 모두 선택된 경우)
    # 여기서는 단순화를 위해 중복된 페이지 범위가 있다면 가장 포괄적인 것을 선택하거나,
    # 또는 가장 구체적인 "상세 요구사항"을 우선. 지금은 일단 나온대로 반환.
    # 더 나은 방법은, '상세 요구사항'이 나왔으면 그게 더 우선순위가 높다고 처리.

    if not target_sections:
        print("경고: LLM 파싱 목차에서 주요 요구사항 섹션을 찾지 못했습니다. 전체 문서를 대상으로 할 수 있습니다.")
        return [{'title': '전체 문서 (목차 분석 실패)', 'start_page': 1, 'end_page': total_pages}]

    return target_sections


def get_toc_raw_text_from_full_text(full_text_with_pages, toc_page_numbers=[2,3]):
    """
    전체 텍스트에서 지정된 목차 페이지들의 텍스트만 추출합니다.
    """
    toc_texts = []
    for page_num in toc_page_numbers:
        # 페이지 마커를 찾아서 해당 페이지의 텍스트를 추출
        # (end_page_num + 1) 마커를 찾아서 그 전까지의 내용을 가져옴
        page_content = extract_text_for_pages(full_text_with_pages, page_num, page_num)
        if page_content:
            # 페이지 마커 제거 (이미 extract_text_for_pages가 마커 다음부터 가져오지만, 혹시 몰라서)
            # 실제로는 페이지 마커 이후의 텍스트만 필요.
            # `extract_text_for_pages`는 마커를 포함해서 반환할 수 있으므로, 마커 이후 내용만 사용.
            marker = f"---PAGE_START_{page_num}---\n"
            if page_content.startswith(marker):
                toc_texts.append(page_content[len(marker):])
            else: # 마커가 없는 경우 (예상치 않음)
                toc_texts.append(page_content)
        else:
            print(f"경고: 목차 페이지로 지정된 {page_num} 페이지에서 텍스트를 찾을 수 없습니다.")

    if not toc_texts:
        return None
    return "\n".join(toc_texts)

# --- 메인 실행 블록 ---
if __name__ == "__main__":
    pdf_file_path = "./data/제주은행 모바일뱅킹 재구축사업.pdf" # 실제 파일 경로
    if not os.path.exists(pdf_file_path):
        print(f"오류: PDF 파일을 찾을 수 없습니다 - {pdf_file_path}")
        exit()

    print("1. PDF 전체 텍스트 추출 중...")
    full_document_text, total_pages = extract_text_with_page_info(pdf_file_path)
    if not full_document_text or total_pages == 0:
        print(f"오류: PDF에서 텍스트를 추출하지 못했거나 페이지 수가 0입니다 ({pdf_file_path}).")
        exit()
    print(f"   PDF 전체 텍스트 추출 완료. 총 {total_pages} 페이지, 전체 텍스트 길이: {len(full_document_text)}자.")

    print("\n2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...")
    # 사용자 PDF 분석 결과 PAGE 2, 3에 목차가 있음
    toc_raw_text = get_toc_raw_text_from_full_text(full_document_text, toc_page_numbers=[2, 3])

    target_sections_for_extraction = []
    if toc_raw_text:
        print(f"   목차 원문 텍스트 추출 완료 (길이: {len(toc_raw_text)}자).")
        print("\n3. LLM을 사용하여 목차(ToC) 파싱 중...")
        parsed_toc_entries = parse_toc_with_llm(toc_raw_text, client, llm_model=LLM_MODEL_FOR_PARSING)

        if parsed_toc_entries:
            print(f"   LLM 목차 파싱 완료. {len(parsed_toc_entries)}개의 목차 항목 식별.")
            print("\n4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...")
            target_sections_for_extraction = get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages)
        else:
            print("   LLM 목차 파싱 실패. 전체 문서를 대상으로 분석합니다.")
            target_sections_for_extraction = [{'title': '전체 문서 (LLM 목차 파싱 실패)', 'start_page': 1, 'end_page': total_pages}]
    else:
        print("   목차 원문 텍스트 추출 실패. 전체 문서를 대상으로 분석합니다.")
        target_sections_for_extraction = [{'title': '전체 문서 (목차 원문 추출 실패)', 'start_page': 1, 'end_page': total_pages}]

    if not target_sections_for_extraction: # 만약을 위해 한번 더 체크
        print("오류: 분석할 대상 섹션을 정의하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    print("\n5. 식별된 주요 섹션으로부터 텍스트 결합 중...")
    all_requirements_related_text_parts = []
    for section_info in target_sections_for_extraction:
        print(f"   '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']}) 텍스트 추출 시도...")
        section_text = extract_text_for_pages(full_document_text, section_info['start_page'], section_info['end_page'])
        if section_text:
            all_requirements_related_text_parts.append(section_text)
            print(f"       '{section_info['title']}' 섹션 텍스트 추출 완료 (길이: {len(section_text)}자).")
        else:
            print(f"       경고: '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']})에서 텍스트를 추출하지 못했습니다.")

    if not all_requirements_related_text_parts:
        print("오류: 주요 요구사항 섹션에서 텍스트를 전혀 추출하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    combined_requirements_text = "\n\n".join(all_requirements_related_text_parts) # 섹션 간 구분을 위해 두 줄바꿈
    print(f"   주요 섹션 텍스트 결합 완료. 총 길이: {len(combined_requirements_text)}자.")

    print("\n6. 결합된 텍스트 초기 분할 중...")
    # chunk_size는 LLM의 context window와 한글 토큰 소모를 고려하여 설정
    initial_chunks = initial_text_split(combined_requirements_text, chunk_size=3500, chunk_overlap=350)
    print(f"   총 {len(initial_chunks)}개의 초기 청크가 생성되었습니다.")

    if not initial_chunks:
        print("오류: 텍스트 분할 결과 청크가 없습니다. 프로그램을 종료합니다.")
        exit()

    print("\n7. LLM을 이용한 개발자 표준 요구사항 상세 추출 시작...")
    extracted_requirements = llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client, llm_model=LLM_MODEL_FOR_PARSING)

    print("\n8. 후처리: ID 중복 제거 및 최종 정렬 (옵션)...")
    # (기존 스크립트의 후처리 로직 사용 또는 필요시 개선)
    unique_requirements_by_id = {}
    processed_requirements = []
    if extracted_requirements:
        for req in extracted_requirements:
            req_id = req.get('id', str(uuid.uuid4())) # ID가 없다면 UUID로 임시 ID
            if req_id not in unique_requirements_by_id:
                unique_requirements_by_id[req_id] = req
            else:
                # ID가 중복될 경우, 설명을 합치거나 페이지 정보를 합치는 등의 로직 추가 가능
                # 여기서는 간단히 첫 번째 것만 유지하거나, 필요에 따라 업데이트
                # 예: 페이지 정보 병합
                existing_req = unique_requirements_by_id[req_id]
                new_pages = req.get('source_pages', [])
                if new_pages:
                    existing_req_pages = existing_req.get('source_pages', [])
                    existing_req['source_pages'] = sorted(list(set(existing_req_pages + new_pages)))
                print(f"중복 ID '{req_id}' 감지. 정보 업데이트 시도.")
        processed_requirements = list(unique_requirements_by_id.values())
    else:
        print("   추출된 요구사항이 없습니다.")

    # 최종 결과 출력 또는 저장
    print(f"\n--- 최종 추출된 고유 요구사항 수: {len(processed_requirements)}개 ---")
    if processed_requirements:
        print("\n--- 추출된 요구사항 미리보기 (상위 3개) ---")
        for i, req_item in enumerate(processed_requirements[:3]): # 'req' 변수명 충돌 피하기 위해 'req_item'으로 변경
            print(json.dumps(req_item, ensure_ascii=False, indent=2))
            print("-" * 30)

        output_filename = "extracted_requirements_final.json"
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(processed_requirements, f, ensure_ascii=False, indent=4)
        print(f"\n추출된 요구사항이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("최종적으로 추출된 요구사항이 없습니다.")

TypeError: OpenAI.__init__() takes 1 positional argument but 2 were given